**Object Counting** (hand)

In [ ]:
%pip install mediapipe
%pip install opencv-python

In [3]:
import cv2
import mediapipe as mp

# Setup MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

# Garis batas Y
line_y = 250
count = 0
prev_y = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)
    h, w, _ = frame.shape

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            cx = int(hand_landmarks.landmark[9].x * w) #pakai center tangan
            cy = int(hand_landmarks.landmark[9].y * h)

            # Gambar titik tengah tangan
            cv2.circle(frame, (cx, cy), 10, (0, 255, 255), -1)

            # Cek crossing line
            if prev_y is not None:
                if prev_y < line_y and cy >= line_y:
                    count += 1
            prev_y = cy

    # Gambar garis batas
    cv2.line(frame, (0, line_y), (w, line_y), (0, 255, 0), 2)
    cv2.putText(frame, f"Count: {count}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    cv2.imshow("Hand Counter", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# Video capture
cap = cv2.VideoCapture(0)

# Garis batas tengah (vertikal)
line_x = 320  # untuk frame 640x480
count_left_to_right = 0
count_right_to_left = 0
prev_x = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            cx = int(hand_landmarks.landmark[9].x * w)
            cy = int(hand_landmarks.landmark[9].y * h)

            # Gambar titik tengah tangan
            cv2.circle(frame, (cx, cy), 10, (0, 255, 255), -1)


            # Counting lintas garis vertikal
            if prev_x is not None:
                if prev_x < line_x and cx >= line_x:
                    count_left_to_right += 1
                elif prev_x > line_x and cx <= line_x:
                    count_right_to_left += 1
            prev_x = cx

    # Gambar garis vertikal
    cv2.line(frame, (line_x, 0), (line_x, h), (255, 0, 0), 2)
    cv2.putText(frame, f"Kiri ke Kanan: {count_left_to_right}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
    cv2.putText(frame, f"Kanan ke Kiri: {count_right_to_left}", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)

    cv2.imshow("Hand Direction Counting", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


#KOMBINASI FACE DAN HAND

In [ ]:
import cv2
import mediapipe as mp
import math

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)

# ini yanggg MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

# Fungsi untuk menggambar tetep
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

# Variabel untuk hand counter
line_y = 300
count = 0
prev_y = None

# Variabel untuk kondisi pintu nyaa
pintu_terbuka = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # tetep pake ini
    result_hand = hands.process(rgb)
    if result_hand.multi_hand_landmarks:
        for hand_landmarks in result_hand.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Mengambil pake tetep 25
            cx = int(hand_landmarks.landmark[9].x * w)
            cy = int(hand_landmarks.landmark[9].y * h)

            # Gambar titik tengah tangan
            cv2.circle(frame, (cx, cy), 10, (0, 255, 255), -1)

            # Cek jika tangan melewati garis
            if prev_y is not None:
                if prev_y < line_y and cy >= line_y:
                    count += 1
            prev_y = cy

    # ini proses pake facemesh 
    is_smiling = False 
    result_face = face_mesh.process(rgb)

    if result_face.multi_face_landmarks:
        for face_landmarks in result_face.multi_face_landmarks:
            # Ambil koordinat sudut bibir kiri kiri 61 sama kanan 291 
            lip_left_corner = face_landmarks.landmark[61]
            lip_right_corner = face_landmarks.landmark[291]
            
            # Konversi ke koordinat piksel
            left_x, left_y = int(lip_left_corner.x * w), int(lip_left_corner.y * h)
            right_x, right_y = int(lip_right_corner.x * w), int(lip_right_corner.y * h)
            
            # jarak koor bibir
            smile_width = math.sqrt((right_x - left_x)**2 + (right_y - left_y)**2)
            
            # Gambar titik disudut koordinat bibir biar keliatan
            cv2.circle(frame, (left_x, left_y), 3, (0, 255, 0), -1)
            cv2.circle(frame, (right_x, right_y), 3, (0, 255, 0), -1)
            
            # Tentukan batas lebar senyum. kasih treshold dikit aja
            SMILE_THRESHOLD = 55 
            if smile_width > SMILE_THRESHOLD:
                is_smiling = True

    # pkae and gabungan nya
    if count >= 7 and is_smiling:
        pintu_terbuka = True
    else:
        pass

    # kasih put text biar jelas 
    if pintu_terbuka:
        cv2.putText(frame, "PINTU TERBUKA", (w // 2 - 200, h // 2),
                    cv2.FONT_HERSHEY_TRIPLEX, 2, (0, 255, 0), 3)

    # garis batas nya tetep pake yang yY
    cv2.line(frame, (0, line_y), (w, line_y), (0, 255, 0), 2)
    
    # Teks Hitungan tanga 
    cv2.putText(frame, f"Count: {count}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
                
    # Teks Status Senyum na
    smile_text = "Senyum: YA" if is_smiling else "Senyum: TIDAK"
    smile_color = (0, 255, 0) if is_smiling else (0, 0, 255)
    cv2.putText(frame, smile_text, (10, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, smile_color, 3)

    cv2.imshow("Pintu Otomatis", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()